In [20]:
"""
SCRIPT INICIALIZACIÓN - eda_completo_con_indices.ipynb
Ejecutar esta celda primero después de reiniciar el kernel
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("INICIALIZANDO NOTEBOOK EDA...")
print("=" * 60)

# ===================================================================
# CARGAR DATASET PROCESADO
# ===================================================================
print("\n1. CARGANDO DATASET PROCESADO...")

rutas_posibles = [
    'data/processed/crypto_stocks_with_indices_complete.csv',
    '../data/processed/crypto_stocks_with_indices_complete.csv',
    '../../data/processed/crypto_stocks_with_indices_complete.csv'
]

df = None
for ruta in rutas_posibles:
    try:
        df = pd.read_csv(ruta)
        df['date'] = pd.to_datetime(df['date'])
        print(f"   ✅ Dataset cargado: {ruta}")
        break
    except:
        continue

if df is None:
    print("   ❌ ERROR: No se pudo cargar el dataset")
else:
    print(f"   📊 Shape: {df.shape}")
    print(f"   📅 Período: {df['date'].min().date()} → {df['date'].max().date()}")
    print(f"   🎯 Activos únicos: {df['asset_name'].nunique()}")
    
    # Verificar columnas clave
    columnas_clave = ['crypto_7_index', 'nasdaq_10_index', 'asset_class']
    columnas_disponibles = [col for col in columnas_clave if col in df.columns]
    print(f"   📋 Columnas clave: {len(columnas_disponibles)}/{len(columnas_clave)}")

# ===================================================================
# CONFIGURAR VISUALIZACIONES
# ===================================================================
print("\n2. CONFIGURANDO VISUALIZACIONES...")

plt.style.use('default')
sns.set_palette("husl")

print("   ✅ Matplotlib y Seaborn configurados")

# ===================================================================
# RESUMEN
# ===================================================================
print("\n" + "=" * 60)
print("LISTO PARA EDA")
print("=" * 60)

if df is not None:
    print(f"Variable 'df' cargada: {df.shape}")
    print("Puedes empezar con el análisis exploratorio")
else:
    print("ERROR: Dataset no cargado - verifica la ruta")

INICIALIZANDO NOTEBOOK EDA...

1. CARGANDO DATASET PROCESADO...
   ✅ Dataset cargado: ../../data/processed/crypto_stocks_with_indices_complete.csv
   📊 Shape: (50109, 18)
   📅 Período: 2012-01-03 → 2024-12-30
   🎯 Activos únicos: 17
   📋 Columnas clave: 3/3

2. CONFIGURANDO VISUALIZACIONES...
   ✅ Matplotlib y Seaborn configurados

LISTO PARA EDA
Variable 'df' cargada: (50109, 18)
Puedes empezar con el análisis exploratorio


In [27]:
print("=" * 80)
print("ANALISIS EXPLORATORIO DE DATOS - CREACION DE INDICES SINTETICOS")
print("=" * 80)

# Configurar visualizaciones
plt.style.use('default')
sns.set_palette("husl")

# ==============================================================================
# PASO 1: CARGAR DATASET BASE (SIN INDICES)
# ==============================================================================
print("\nPASO 1: CARGANDO DATASET BASE...")

file_path = r'c:\Users\USER\Desktop\Thepower dataanalytic\Proyectos The power\proyecto-final\data\processed\crypto_stocks_clean_raw.csv'

df = pd.read_csv(file_path)
df['date'] = pd.to_datetime(df['date'])

print(f"\nDataset base cargado: {len(df):,} filas x {len(df.columns)} columnas")
print(f"Periodo: {df['date'].min().date()} -> {df['date'].max().date()}")
print(f"Activos unicos: {df['asset_name'].nunique()}")

# Ver estructura
print(f"\nColumnas del dataset base:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. {col}")

# Separar cryptos y stocks
cryptos = df[df['asset_class'] == 'crypto']['asset_name'].unique()
stocks = df[df['asset_class'] == 'stock']['asset_name'].unique()

print(f"\nCriptomonedas ({len(cryptos)}): {list(cryptos)}")
print(f"Acciones ({len(stocks)}): {list(stocks)}")

print("\nDataset base listo para crear indices")

ANALISIS EXPLORATORIO DE DATOS - CREACION DE INDICES SINTETICOS

PASO 1: CARGANDO DATASET BASE...

Dataset base cargado: 50,109 filas x 9 columnas
Periodo: 2012-01-03 -> 2024-12-30
Activos unicos: 17

Columnas del dataset base:
  1. date
  2. close
  3. high
  4. low
  5. open
  6. volume
  7. asset_name
  8. asset_class
  9. symbol

Criptomonedas (7): ['BITCOIN', 'ETHEREUM', 'BINANCECOIN', 'CARDANO', 'SOLANA', 'DOGECOIN', 'POLKADOT']
Acciones (10): ['AAPL', 'TSLA', 'MSFT', 'AMZN', 'GOOGL', 'NVDA', 'META', 'INTC', 'AMD', 'QCOM']

Dataset base listo para crear indices


In [28]:
# ==============================================================================
# PASO 2: CREAR NASDAQ-10 INDEX
# ==============================================================================

print("\n" + "=" * 80)
print("PASO 2: CREANDO NASDAQ-10 INDEX")
print("=" * 80)

# Filtrar solo stocks
stock_data = df[df['asset_class'] == 'stock'].copy()

# Verificar fechas de inicio por stock
print("\nFechas de inicio por stock:")
stock_start_dates = {}
for stock in stocks:
    start_date = stock_data[stock_data['asset_name'] == stock]['date'].min()
    stock_start_dates[stock] = start_date
    print(f"  {stock}: {start_date.date()}")

# Fecha comun mas temprana (todas las acciones disponibles)
earliest_common_date = max(stock_start_dates.values())
print(f"\nFecha comun inicio: {earliest_common_date.date()}")
print(f"  (espera a que todas las {len(stocks)} acciones tengan datos)")

# Filtrar desde fecha comun
stock_data_completo = stock_data[stock_data['date'] >= earliest_common_date].copy()

# Crear matriz de precios
stock_pivot = stock_data_completo.pivot_table(
    values='close',
    index='date',
    columns='asset_name'
).fillna(method='ffill')

print(f"\nMatriz de precios: {stock_pivot.shape}")
print(f"  Stocks: {list(stock_pivot.columns)}")
print(f"  Periodo: {stock_pivot.index.min().date()} -> {stock_pivot.index.max().date()}")
print(f"  Dias: {len(stock_pivot):,}")

# Calcular retornos diarios
stock_returns = stock_pivot.pct_change().fillna(0)

# Equally weighted - promedio simple
stock_avg_returns = stock_returns.mean(axis=1)

# Construir indice base 100
nasdaq_index = [100]
for ret in stock_avg_returns[1:]:
    nasdaq_index.append(nasdaq_index[-1] * (1 + ret))

nasdaq_index = pd.Series(nasdaq_index, index=stock_pivot.index)

# Medias moviles
nasdaq_ma8 = nasdaq_index.rolling(window=8).mean()
nasdaq_ma21 = nasdaq_index.rolling(window=21).mean()

# Performance
nasdaq_performance = ((nasdaq_index.iloc[-1]/nasdaq_index.iloc[0])-1)*100
años_nasdaq = (nasdaq_index.index.max() - nasdaq_index.index.min()).days / 365.25

print(f"\nNASDAQ-10 INDEX CREADO:")
print(f"  Valor inicial: {nasdaq_index.iloc[0]:.2f}")
print(f"  Valor final: {nasdaq_index.iloc[-1]:.2f}")
print(f"  Performance: {nasdaq_performance:.1f}%")
print(f"  Años: {años_nasdaq:.1f}")
print(f"  MA8 disponible desde: {nasdaq_ma8.dropna().index.min().date()}")
print(f"  MA21 disponible desde: {nasdaq_ma21.dropna().index.min().date()}")


PASO 2: CREANDO NASDAQ-10 INDEX

Fechas de inicio por stock:
  AAPL: 2012-01-03
  TSLA: 2012-01-03
  MSFT: 2012-01-03
  AMZN: 2012-01-03
  GOOGL: 2012-01-03
  NVDA: 2012-01-03
  META: 2012-05-18
  INTC: 2012-01-03
  AMD: 2012-01-03
  QCOM: 2012-01-03

Fecha comun inicio: 2012-05-18
  (espera a que todas las 10 acciones tengan datos)

Matriz de precios: (3174, 10)
  Stocks: ['AAPL', 'AMD', 'AMZN', 'GOOGL', 'INTC', 'META', 'MSFT', 'NVDA', 'QCOM', 'TSLA']
  Periodo: 2012-05-18 -> 2024-12-30
  Dias: 3,174

NASDAQ-10 INDEX CREADO:
  Valor inicial: 100.00
  Valor final: 3464.05
  Performance: 3364.1%
  Años: 12.6
  MA8 disponible desde: 2012-05-30
  MA21 disponible desde: 2012-06-18


In [29]:
# ==============================================================================
# PASO 3: CREAR CRYPTO-7 INDEX
# ==============================================================================

print("\n" + "=" * 80)
print("PASO 3: CREANDO CRYPTO-7 INDEX")
print("=" * 80)

# Filtrar solo cryptos
crypto_data = df[df['asset_class'] == 'crypto'].copy()

# Verificar fechas de inicio
print("\nFechas de inicio por crypto:")
crypto_start_dates = {}
for crypto in cryptos:
    start_date = crypto_data[crypto_data['asset_name'] == crypto]['date'].min()
    crypto_start_dates[crypto] = start_date
    print(f"  {crypto}: {start_date.date()}")

# Fecha comun (todas las cryptos disponibles)
crypto_common_date = max(crypto_start_dates.values())
print(f"\nFecha comun inicio: {crypto_common_date.date()}")
print(f"  (espera a que todas las {len(cryptos)} cryptos tengan datos)")

# Filtrar desde fecha comun
crypto_data_completo = crypto_data[crypto_data['date'] >= crypto_common_date].copy()

# Crear matriz de precios
crypto_pivot = crypto_data_completo.pivot_table(
    values='close',
    index='date',
    columns='asset_name'
).fillna(method='ffill')

print(f"\nMatriz de precios: {crypto_pivot.shape}")
print(f"  Cryptos: {list(crypto_pivot.columns)}")
print(f"  Periodo: {crypto_pivot.index.min().date()} -> {crypto_pivot.index.max().date()}")
print(f"  Dias: {len(crypto_pivot):,}")

# Calcular retornos
crypto_returns = crypto_pivot.pct_change().fillna(0)

# Equally weighted
crypto_avg_returns = crypto_returns.mean(axis=1)

# Construir indice base 100
crypto_index = [100]
for ret in crypto_avg_returns[1:]:
    crypto_index.append(crypto_index[-1] * (1 + ret))

crypto_index = pd.Series(crypto_index, index=crypto_pivot.index)

# Medias moviles
crypto_ma8 = crypto_index.rolling(window=8).mean()
crypto_ma21 = crypto_index.rolling(window=21).mean()

# Performance
crypto_performance = ((crypto_index.iloc[-1]/crypto_index.iloc[0])-1)*100
años_crypto = (crypto_index.index.max() - crypto_index.index.min()).days / 365.25

print(f"\nCRYPTO-7 INDEX CREADO:")
print(f"  Valor inicial: {crypto_index.iloc[0]:.2f}")
print(f"  Valor final: {crypto_index.iloc[-1]:.2f}")
print(f"  Performance: {crypto_performance:.1f}%")
print(f"  Años: {años_crypto:.1f}")
print(f"  MA8 disponible desde: {crypto_ma8.dropna().index.min().date()}")
print(f"  MA21 disponible desde: {crypto_ma21.dropna().index.min().date()}")


PASO 3: CREANDO CRYPTO-7 INDEX

Fechas de inicio por crypto:
  BITCOIN: 2014-09-17
  ETHEREUM: 2017-11-09
  BINANCECOIN: 2017-11-09
  CARDANO: 2017-11-09
  SOLANA: 2020-04-10
  DOGECOIN: 2017-11-09
  POLKADOT: 2020-08-20

Fecha comun inicio: 2020-08-20
  (espera a que todas las 7 cryptos tengan datos)

Matriz de precios: (1594, 7)
  Cryptos: ['BINANCECOIN', 'BITCOIN', 'CARDANO', 'DOGECOIN', 'ETHEREUM', 'POLKADOT', 'SOLANA']
  Periodo: 2020-08-20 -> 2024-12-30
  Dias: 1,594

CRYPTO-7 INDEX CREADO:
  Valor inicial: 100.00
  Valor final: 4240.62
  Performance: 4140.6%
  Años: 4.4
  MA8 disponible desde: 2020-08-27
  MA21 disponible desde: 2020-09-09


In [30]:
# ==============================================================================
# PASO 4: FUSIONAR INDICES CON DATASET ORIGINAL
# ==============================================================================

print("\n" + "=" * 80)
print("PASO 4: FUSIONANDO INDICES CON DATASET ORIGINAL")
print("=" * 80)

# Crear dataset de indices
todas_las_fechas = df['date'].unique()
todas_las_fechas = pd.Series(todas_las_fechas).sort_values()

dataset_indices = pd.DataFrame({
    'date': todas_las_fechas
})

print(f"\nFechas base: {len(dataset_indices):,}")

# Agregar NASDAQ-10
nasdaq_df = pd.DataFrame({
    'date': nasdaq_index.index,
    'nasdaq_10_index': nasdaq_index.values,
    'nasdaq_10_ma8': nasdaq_ma8.values,
    'nasdaq_10_ma21': nasdaq_ma21.values
})

dataset_indices = pd.merge(dataset_indices, nasdaq_df, on='date', how='left')
print(f"NASDAQ-10 agregado: {dataset_indices['nasdaq_10_index'].notna().sum():,} fechas")

# Agregar CRYPTO-7
crypto_df = pd.DataFrame({
    'date': crypto_index.index,
    'crypto_7_index': crypto_index.values,
    'crypto_7_ma8': crypto_ma8.values,
    'crypto_7_ma21': crypto_ma21.values
})

dataset_indices = pd.merge(dataset_indices, crypto_df, on='date', how='left')
print(f"CRYPTO-7 agregado: {dataset_indices['crypto_7_index'].notna().sum():,} fechas")

# Calcular metricas derivadas
dataset_indices['nasdaq_10_returns'] = dataset_indices['nasdaq_10_index'].pct_change()
dataset_indices['crypto_7_returns'] = dataset_indices['crypto_7_index'].pct_change()
dataset_indices['crypto_nasdaq_ratio'] = dataset_indices['crypto_7_index'] / dataset_indices['nasdaq_10_index']

print(f"\nMetricas derivadas calculadas")

# Merge final con dataset original
df_final = pd.merge(df, dataset_indices, on='date', how='left')

print(f"\nDATASET FINAL:")
print(f"  Shape: {df_final.shape}")
print(f"  Periodo: {df_final['date'].min().date()} -> {df_final['date'].max().date()}")
print(f"  Columnas totales: {len(df_final.columns)}")

# Mostrar nuevas columnas
nuevas_cols = [col for col in df_final.columns if col not in df.columns]
print(f"\nNuevas columnas agregadas ({len(nuevas_cols)}):")
for col in nuevas_cols:
    valores_validos = df_final[col].notna().sum()
    pct = (valores_validos / len(df_final)) * 100
    print(f"  {col}: {valores_validos:,} valores ({pct:.1f}%)")


PASO 4: FUSIONANDO INDICES CON DATASET ORIGINAL

Fechas base: 4,438
NASDAQ-10 agregado: 3,174 fechas
CRYPTO-7 agregado: 1,594 fechas

Metricas derivadas calculadas

DATASET FINAL:
  Shape: (50109, 18)
  Periodo: 2012-01-03 -> 2024-12-30
  Columnas totales: 18

Nuevas columnas agregadas (9):
  nasdaq_10_index: 43,794 valores (87.4%)
  nasdaq_10_ma8: 43,724 valores (87.3%)
  nasdaq_10_ma21: 43,594 valores (87.0%)
  crypto_7_index: 22,128 valores (44.2%)
  crypto_7_ma8: 22,029 valores (44.0%)
  crypto_7_ma21: 21,858 valores (43.6%)
  nasdaq_10_returns: 49,244 valores (98.3%)
  crypto_7_returns: 22,111 valores (44.1%)
  crypto_nasdaq_ratio: 18,649 valores (37.2%)


In [31]:
# ==============================================================================
# PASO 5: OPTIMIZAR (FORWARD FILL) Y GUARDAR
# ==============================================================================

print("\n" + "=" * 80)
print("PASO 5: OPTIMIZANDO Y GUARDANDO DATASET FINAL")
print("=" * 80)

# Aplicar forward fill para completar gaps en indices
print("\nAplicando forward fill inteligente...")

# NASDAQ desde su fecha de inicio
nasdaq_start = df_final[df_final['nasdaq_10_index'].notna()]['date'].min()
mask_nasdaq = df_final['date'] >= nasdaq_start

for i in range(3):  # Multiple passes
    df_final.loc[mask_nasdaq, 'nasdaq_10_index'] = df_final.loc[mask_nasdaq, 'nasdaq_10_index'].fillna(method='ffill')
    df_final.loc[mask_nasdaq, 'nasdaq_10_ma8'] = df_final.loc[mask_nasdaq, 'nasdaq_10_ma8'].fillna(method='ffill')
    df_final.loc[mask_nasdaq, 'nasdaq_10_ma21'] = df_final.loc[mask_nasdaq, 'nasdaq_10_ma21'].fillna(method='ffill')

# Backward fill inicio si necesario
df_final.loc[mask_nasdaq, 'nasdaq_10_index'] = df_final.loc[mask_nasdaq, 'nasdaq_10_index'].fillna(method='bfill')
df_final.loc[mask_nasdaq, 'nasdaq_10_ma8'] = df_final.loc[mask_nasdaq, 'nasdaq_10_ma8'].fillna(method='bfill')
df_final.loc[mask_nasdaq, 'nasdaq_10_ma21'] = df_final.loc[mask_nasdaq, 'nasdaq_10_ma21'].fillna(method='bfill')

print(f"  NASDAQ-10 optimizado")

# CRYPTO desde su fecha de inicio
crypto_start = df_final[df_final['crypto_7_index'].notna()]['date'].min()
mask_crypto = df_final['date'] >= crypto_start

for i in range(3):
    df_final.loc[mask_crypto, 'crypto_7_index'] = df_final.loc[mask_crypto, 'crypto_7_index'].fillna(method='ffill')
    df_final.loc[mask_crypto, 'crypto_7_ma8'] = df_final.loc[mask_crypto, 'crypto_7_ma8'].fillna(method='ffill')
    df_final.loc[mask_crypto, 'crypto_7_ma21'] = df_final.loc[mask_crypto, 'crypto_7_ma21'].fillna(method='ffill')

df_final.loc[mask_crypto, 'crypto_7_index'] = df_final.loc[mask_crypto, 'crypto_7_index'].fillna(method='bfill')
df_final.loc[mask_crypto, 'crypto_7_ma8'] = df_final.loc[mask_crypto, 'crypto_7_ma8'].fillna(method='bfill')
df_final.loc[mask_crypto, 'crypto_7_ma21'] = df_final.loc[mask_crypto, 'crypto_7_ma21'].fillna(method='bfill')

print(f"  CRYPTO-7 optimizado")

# Recalcular ratio
mask_both = df_final['crypto_7_index'].notna() & df_final['nasdaq_10_index'].notna()
df_final.loc[mask_both, 'crypto_nasdaq_ratio'] = (
    df_final.loc[mask_both, 'crypto_7_index'] / df_final.loc[mask_both, 'nasdaq_10_index']
)

print(f"\nRatio recalculado: {df_final['crypto_nasdaq_ratio'].notna().sum():,} valores")

# GUARDAR archivo final
output_path = r'c:\Users\USER\Desktop\Thepower dataanalytic\Proyectos The power\proyecto-final\data\processed\crypto_stocks_with_indices_complete.csv'

df_final.to_csv(output_path, index=False)

import os
file_size = os.path.getsize(output_path) / 1024 / 1024

print(f"\nARCHIVO GUARDADO:")
print(f"  Ubicacion: {output_path}")
print(f"  Tamaño: {file_size:.2f} MB")
print(f"  Filas: {len(df_final):,}")
print(f"  Columnas: {len(df_final.columns)}")

print("\nPROCESO COMPLETO - INDICES CREADOS Y FUSIONADOS")


PASO 5: OPTIMIZANDO Y GUARDANDO DATASET FINAL

Aplicando forward fill inteligente...
  NASDAQ-10 optimizado
  CRYPTO-7 optimizado

Ratio recalculado: 22,128 valores

ARCHIVO GUARDADO:
  Ubicacion: c:\Users\USER\Desktop\Thepower dataanalytic\Proyectos The power\proyecto-final\data\processed\crypto_stocks_with_indices_complete.csv
  Tamaño: 11.05 MB
  Filas: 50,109
  Columnas: 18

PROCESO COMPLETO - INDICES CREADOS Y FUSIONADOS


In [32]:
# ==============================================================================
# ANALISIS: COMPARACION PERIODO COMUN
# ==============================================================================

print("\n" + "=" * 80)
print("ANALISIS COMPARATIVO PERIODO COMUN")
print("=" * 80)

# Extraer indices como series
nasdaq_index_final = df_final[df_final['nasdaq_10_index'].notna()].drop_duplicates('date').sort_values('date')
nasdaq_series = pd.Series(nasdaq_index_final['nasdaq_10_index'].values, index=nasdaq_index_final['date'])

crypto_index_final = df_final[df_final['crypto_7_index'].notna()].drop_duplicates('date').sort_values('date')
crypto_series = pd.Series(crypto_index_final['crypto_7_index'].values, index=crypto_index_final['date'])

# Periodo comun
fechas_comunes = crypto_series.index.intersection(nasdaq_series.index)

if len(fechas_comunes) > 0:
    nasdaq_comun = nasdaq_series.loc[fechas_comunes]
    crypto_comun = crypto_series.loc[fechas_comunes]
    
    nasdaq_perf_comun = ((nasdaq_comun.iloc[-1]/nasdaq_comun.iloc[0])-1)*100
    crypto_perf_comun = ((crypto_comun.iloc[-1]/crypto_comun.iloc[0])-1)*100
    
    print(f"\nPeriodo comun: {fechas_comunes[0].date()} -> {fechas_comunes[-1].date()}")
    print(f"Dias: {len(fechas_comunes):,} ({len(fechas_comunes)/365:.1f} años)")
    
    print(f"\nCRYPTO-7: {crypto_perf_comun:.1f}%")
    print(f"NASDAQ-10: {nasdaq_perf_comun:.1f}%")
    
    if crypto_perf_comun > nasdaq_perf_comun:
        diferencia = crypto_perf_comun - nasdaq_perf_comun
        ratio = crypto_perf_comun / nasdaq_perf_comun if nasdaq_perf_comun > 0 else 0
        print(f"\nGANADOR: CRYPTO-7 (+{diferencia:.1f}pp, {ratio:.1f}x)")
    else:
        diferencia = nasdaq_perf_comun - crypto_perf_comun
        ratio = nasdaq_perf_comun / crypto_perf_comun if crypto_perf_comun > 0 else 0
        print(f"\nGANADOR: NASDAQ-10 (+{diferencia:.1f}pp, {ratio:.1f}x)")


ANALISIS COMPARATIVO PERIODO COMUN

Periodo comun: 2020-08-20 -> 2024-12-30
Dias: 1,594 (4.4 años)

CRYPTO-7: 4140.6%
NASDAQ-10: 144.2%

GANADOR: CRYPTO-7 (+3996.4pp, 28.7x)


In [34]:
# ==============================================================================
# AUDITORIA: ESTADO DE NaNs EN DATASET FINAL
# ==============================================================================

print("\n" + "=" * 80)
print("AUDITORIA DE NaNs - DATASET FINAL")
print("=" * 80)

# Columnas de indices a verificar
cols_indices = [
    'nasdaq_10_index', 'nasdaq_10_ma8', 'nasdaq_10_ma21', 'nasdaq_10_returns',
    'crypto_7_index', 'crypto_7_ma8', 'crypto_7_ma21', 'crypto_7_returns',
    'crypto_nasdaq_ratio'
]

print(f"\nDIMENSIONES DATASET:")
print(f"  Filas: {len(df_final):,}")
print(f"  Columnas: {len(df_final.columns)}")

# Crear reporte de NaNs
print(f"\nREPORTE DE NaNs POR COLUMNA:")
print("-" * 80)
print(f"{'Columna':<25} {'NaNs':>10} {'%':>8} {'Validos':>10} {'%':>8}")
print("-" * 80)

for col in cols_indices:
    if col in df_final.columns:
        nans = df_final[col].isnull().sum()
        pct_nan = (nans / len(df_final)) * 100
        validos = df_final[col].notna().sum()
        pct_val = (validos / len(df_final)) * 100
        
        print(f"{col:<25} {nans:>10,} {pct_nan:>7.2f}% {validos:>10,} {pct_val:>7.2f}%")

# Analisis detallado por periodo
print(f"\n" + "=" * 80)
print("ANALISIS DE NaNs POR PERIODO")
print("=" * 80)

# NASDAQ
nasdaq_start = df_final[df_final['nasdaq_10_index'].notna()]['date'].min()
nasdaq_nans_antes = df_final[df_final['date'] < nasdaq_start]['nasdaq_10_index'].isnull().sum()
nasdaq_nans_despues = df_final[df_final['date'] >= nasdaq_start]['nasdaq_10_index'].isnull().sum()

print(f"\nNASDAQ-10 INDEX:")
print(f"  Inicio datos: {nasdaq_start.date()}")
print(f"  NaNs ANTES del inicio: {nasdaq_nans_antes:,} (correcto - no existia)")
print(f"  NaNs DESPUES del inicio: {nasdaq_nans_despues:,}", end="")
if nasdaq_nans_despues == 0:
    print(" ✓ PERFECTO")
else:
    print(" ⚠ REVISAR")

# CRYPTO
crypto_start = df_final[df_final['crypto_7_index'].notna()]['date'].min()
crypto_nans_antes = df_final[df_final['date'] < crypto_start]['crypto_7_index'].isnull().sum()
crypto_nans_despues = df_final[df_final['date'] >= crypto_start]['crypto_7_index'].isnull().sum()

print(f"\nCRYPTO-7 INDEX:")
print(f"  Inicio datos: {crypto_start.date()}")
print(f"  NaNs ANTES del inicio: {crypto_nans_antes:,} (correcto - no existia)")
print(f"  NaNs DESPUES del inicio: {crypto_nans_despues:,}", end="")
if crypto_nans_despues == 0:
    print(" ✓ PERFECTO")
else:
    print(" ⚠ REVISAR")

# Resumen general
total_nans = sum([df_final[col].isnull().sum() for col in cols_indices if col in df_final.columns])
total_cells = len(df_final) * len(cols_indices)

print(f"\n" + "=" * 80)
print(f"RESUMEN GENERAL:")
print(f"  Total NaNs en indices: {total_nans:,}")
print(f"  Total celdas: {total_cells:,}")
print(f"  % NaNs global: {(total_nans/total_cells)*100:.2f}%")

# Verificacion final
if nasdaq_nans_despues == 0 and crypto_nans_despues == 0:
    print(f"\n✓ DATASET OPTIMO: Sin NaNs despues de fechas de inicio")
else:
    print(f"\n⚠ ATENCION: Hay NaNs que deberian estar rellenados")
    
print("=" * 80)


AUDITORIA DE NaNs - DATASET FINAL

DIMENSIONES DATASET:
  Filas: 50,109
  Columnas: 18

REPORTE DE NaNs POR COLUMNA:
--------------------------------------------------------------------------------
Columna                         NaNs        %    Validos        %
--------------------------------------------------------------------------------
nasdaq_10_index                  855    1.71%     49,254   98.29%
nasdaq_10_ma8                    855    1.71%     49,254   98.29%
nasdaq_10_ma21                   855    1.71%     49,254   98.29%
nasdaq_10_returns                865    1.73%     49,244   98.27%
crypto_7_index                27,981   55.84%     22,128   44.16%
crypto_7_ma8                  27,981   55.84%     22,128   44.16%
crypto_7_ma21                 27,981   55.84%     22,128   44.16%
crypto_7_returns              27,998   55.87%     22,111   44.13%
crypto_nasdaq_ratio           27,981   55.84%     22,128   44.16%

ANALISIS DE NaNs POR PERIODO

NASDAQ-10 INDEX:
  Inicio dat

In [35]:
# ==============================================================================
# ANALISIS: PERFORMANCE CRYPTOS INDIVIDUALES vs CRYPTO-7
# ==============================================================================

print("\n" + "=" * 80)
print("PERFORMANCE CRYPTOS INDIVIDUALES vs CRYPTO-7 INDEX")
print("=" * 80)

# Extraer crypto index como serie
crypto_index_data = df_final[df_final['crypto_7_index'].notna()].drop_duplicates('date').sort_values('date')
crypto_index = pd.Series(crypto_index_data['crypto_7_index'].values, index=crypto_index_data['date'])

crypto_start = crypto_index.index.min()
crypto_performance_total = ((crypto_index.iloc[-1]/crypto_index.iloc[0])-1)*100

# Filtrar cryptos individuales
crypto_data_individual = df_final[df_final['asset_class'] == 'crypto'].copy()
cryptos = crypto_data_individual['asset_name'].unique()

print(f"\nPeriodo: {crypto_start.date()} -> {crypto_index.index.max().date()}")
print(f"CRYPTO-7 Index: {crypto_performance_total:.1f}%")
print(f"\nCOMPARACION INDIVIDUAL:\n")

# Crear pivot
crypto_pivot = crypto_data_individual.pivot_table(
    values='close',
    index='date',
    columns='asset_name'
).loc[crypto_start:]

crypto_individual_results = []

for crypto in cryptos:
    if crypto in crypto_pivot.columns:
        precio_inicio = crypto_pivot[crypto].iloc[0]
        precio_final = crypto_pivot[crypto].iloc[-1]
        performance = ((precio_final / precio_inicio) - 1) * 100
        vs_index = performance - crypto_performance_total
        
        crypto_individual_results.append({
            'crypto': crypto,
            'precio_inicio': precio_inicio,
            'precio_final': precio_final,
            'performance': performance,
            'vs_index': vs_index
        })
        
        simbolo = "↑" if performance > crypto_performance_total else "↓"
        print(f"{simbolo} {crypto:<15}: ${precio_inicio:>8.2f} -> ${precio_final:>10.2f}")
        print(f"   {performance:>+8.1f}% (vs indice: {vs_index:>+6.1f}%)")

# Ranking
crypto_ranking = pd.DataFrame(crypto_individual_results).sort_values('performance', ascending=False)

print(f"\nRANKING CRYPTOS:")
for idx, row in crypto_ranking.iterrows():
    pos = crypto_ranking.index.get_loc(idx) + 1
    print(f"{pos}. {row['crypto']:<15}: {row['performance']:>+8.1f}%")

ganadoras = len(crypto_ranking[crypto_ranking['vs_index'] > 0])
print(f"\nCryptos que superaron al indice: {ganadoras}/{len(cryptos)}")


PERFORMANCE CRYPTOS INDIVIDUALES vs CRYPTO-7 INDEX

Periodo: 2020-08-20 -> 2024-12-30
CRYPTO-7 Index: 4140.6%

COMPARACION INDIVIDUAL:

↓ BITCOIN        : $11878.37 -> $  92643.21
     +679.9% (vs indice: -3460.7%)
↓ ETHEREUM       : $  416.44 -> $   3356.39
     +706.0% (vs indice: -3434.6%)
↓ BINANCECOIN    : $   23.09 -> $    704.15
    +2949.2% (vs indice: -1191.4%)
↓ CARDANO        : $    0.13 -> $      0.86
     +543.5% (vs indice: -3597.1%)
↑ SOLANA         : $    3.21 -> $    191.03
    +5854.2% (vs indice: +1713.6%)
↑ DOGECOIN       : $    0.00 -> $      0.31
    +8963.7% (vs indice: +4823.1%)
↓ POLKADOT       : $    2.90 -> $      6.70
     +130.9% (vs indice: -4009.7%)

RANKING CRYPTOS:
1. DOGECOIN       :  +8963.7%
2. SOLANA         :  +5854.2%
3. BINANCECOIN    :  +2949.2%
4. ETHEREUM       :   +706.0%
5. BITCOIN        :   +679.9%
6. CARDANO        :   +543.5%
7. POLKADOT       :   +130.9%

Cryptos que superaron al indice: 2/7


In [36]:
# ==============================================================================
# ANALISIS: PERFORMANCE STOCKS INDIVIDUALES vs NASDAQ-10
# ==============================================================================

print("\n" + "=" * 80)
print("PERFORMANCE STOCKS INDIVIDUALES vs NASDAQ-10 INDEX")
print("=" * 80)

# Extraer nasdaq index como serie
nasdaq_index_data = df_final[df_final['nasdaq_10_index'].notna()].drop_duplicates('date').sort_values('date')
nasdaq_index = pd.Series(nasdaq_index_data['nasdaq_10_index'].values, index=nasdaq_index_data['date'])

nasdaq_start = nasdaq_index.index.min()
nasdaq_performance_total = ((nasdaq_index.iloc[-1]/nasdaq_index.iloc[0])-1)*100

# Filtrar stocks individuales
stock_data_individual = df_final[df_final['asset_class'] == 'stock'].copy()
stocks = stock_data_individual['asset_name'].unique()

print(f"\nPeriodo: {nasdaq_start.date()} -> {nasdaq_index.index.max().date()}")
print(f"NASDAQ-10 Index: {nasdaq_performance_total:.1f}%")
print(f"\nCOMPARACION INDIVIDUAL:\n")

# Crear pivot
stock_pivot = stock_data_individual.pivot_table(
    values='close',
    index='date',
    columns='asset_name'
).loc[nasdaq_start:]

stock_individual_results = []

for stock in stocks:
    if stock in stock_pivot.columns:
        precio_inicio = stock_pivot[stock].iloc[0]
        precio_final = stock_pivot[stock].iloc[-1]
        performance = ((precio_final / precio_inicio) - 1) * 100
        vs_index = performance - nasdaq_performance_total
        
        stock_individual_results.append({
            'stock': stock,
            'precio_inicio': precio_inicio,
            'precio_final': precio_final,
            'performance': performance,
            'vs_index': vs_index
        })
        
        simbolo = "↑" if performance > nasdaq_performance_total else "↓"
        print(f"{simbolo} {stock:<25}: ${precio_inicio:>8.2f} -> ${precio_final:>10.2f}")
        print(f"   {performance:>+8.1f}% (vs indice: {vs_index:>+6.1f}%)")

# Ranking
stock_ranking = pd.DataFrame(stock_individual_results).sort_values('performance', ascending=False)

print(f"\nRANKING STOCKS:")
for idx, row in stock_ranking.iterrows():
    pos = stock_ranking.index.get_loc(idx) + 1
    print(f"{pos}. {row['stock']:<25}: {row['performance']:>+8.1f}%")

ganadoras = len(stock_ranking[stock_ranking['vs_index'] > 0])
print(f"\nStocks que superaron al indice: {ganadoras}/{len(stocks)}")


PERFORMANCE STOCKS INDIVIDUALES vs NASDAQ-10 INDEX

Periodo: 2012-05-18 -> 2024-12-30
NASDAQ-10 Index: 3364.1%

COMPARACION INDIVIDUAL:

↓ AAPL                     : $   15.92 -> $    251.31
    +1478.4% (vs indice: -1885.7%)
↑ TSLA                     : $    1.84 -> $    417.41
   +22618.3% (vs indice: +19254.2%)
↓ MSFT                     : $   23.27 -> $    422.51
    +1715.7% (vs indice: -1648.3%)
↓ AMZN                     : $   10.69 -> $    221.30
    +1969.7% (vs indice: -1394.4%)
↓ GOOGL                    : $   14.92 -> $    190.62
    +1177.4% (vs indice: -2186.6%)
↑ NVDA                     : $    0.28 -> $    137.46
   +49542.6% (vs indice: +46178.5%)
↓ META                     : $   38.00 -> $    589.86
    +1452.4% (vs indice: -1911.6%)
↓ INTC                     : $   18.29 -> $     19.82
       +8.3% (vs indice: -3355.7%)
↓ AMD                      : $    6.01 -> $    122.44
    +1937.3% (vs indice: -1426.8%)
↓ QCOM                     : $   39.04 -> $    151.97
     

In [37]:
# ==============================================================================
# ANALISIS: VOLATILIDAD Y RIESGO
# ==============================================================================

print("\n" + "=" * 80)
print("ANALISIS DE VOLATILIDAD - RIESGO vs RETORNO")
print("=" * 80)

# Extraer indices
nasdaq_index_data = df_final[df_final['nasdaq_10_index'].notna()].drop_duplicates('date').sort_values('date')
nasdaq_index = pd.Series(nasdaq_index_data['nasdaq_10_index'].values, index=nasdaq_index_data['date'])

crypto_index_data = df_final[df_final['crypto_7_index'].notna()].drop_duplicates('date').sort_values('date')
crypto_index = pd.Series(crypto_index_data['crypto_7_index'].values, index=crypto_index_data['date'])

# Calcular returns diarios
crypto_returns = crypto_index.pct_change().dropna()
nasdaq_returns = nasdaq_index.pct_change().dropna()

# Volatilidad anualizada (252 dias trading)
crypto_vol = crypto_returns.std() * np.sqrt(252) * 100
nasdaq_vol = nasdaq_returns.std() * np.sqrt(252) * 100

print(f"\nVOLATILIDAD ANUALIZADA:")
print(f"CRYPTO-7:  {crypto_vol:.1f}%")
print(f"NASDAQ-10: {nasdaq_vol:.1f}%")
print(f"Ratio: {crypto_vol/nasdaq_vol:.1f}x")

# Performance anualizada
años_crypto = (crypto_index.index.max() - crypto_index.index.min()).days / 365.25
años_nasdaq = (nasdaq_index.index.max() - nasdaq_index.index.min()).days / 365.25

crypto_perf = ((crypto_index.iloc[-1]/crypto_index.iloc[0])-1)*100
nasdaq_perf = ((nasdaq_index.iloc[-1]/nasdaq_index.iloc[0])-1)*100

crypto_ret_anual = crypto_perf / años_crypto
nasdaq_ret_anual = nasdaq_perf / años_nasdaq

# Sharpe Ratio simplificado
crypto_sharpe = crypto_ret_anual / crypto_vol
nasdaq_sharpe = nasdaq_ret_anual / nasdaq_vol

print(f"\nSHARPE RATIO (simplificado):")
print(f"CRYPTO-7:  {crypto_sharpe:.2f}")
print(f"NASDAQ-10: {nasdaq_sharpe:.2f}")

mejor_sharpe = "CRYPTO-7" if crypto_sharpe > nasdaq_sharpe else "NASDAQ-10"
print(f"\nMejor ratio riesgo/retorno: {mejor_sharpe}")

# Drawdown maximo
def calcular_drawdown(returns_series):
    cumulative = (1 + returns_series).cumprod()
    rolling_max = cumulative.expanding().max()
    drawdown = ((cumulative - rolling_max) / rolling_max) * 100
    return drawdown.min()

crypto_dd = calcular_drawdown(crypto_returns)
nasdaq_dd = calcular_drawdown(nasdaq_returns)

print(f"\nDRAWDOWN MAXIMO:")
print(f"CRYPTO-7:  {crypto_dd:.1f}%")
print(f"NASDAQ-10: {nasdaq_dd:.1f}%")


ANALISIS DE VOLATILIDAD - RIESGO vs RETORNO

VOLATILIDAD ANUALIZADA:
CRYPTO-7:  68.0%
NASDAQ-10: 22.5%
Ratio: 3.0x

SHARPE RATIO (simplificado):
CRYPTO-7:  13.95
NASDAQ-10: 11.83

Mejor ratio riesgo/retorno: CRYPTO-7

DRAWDOWN MAXIMO:
CRYPTO-7:  -82.7%
NASDAQ-10: -48.9%


In [38]:
# ==============================================================================
# ANALISIS: CORRELACIONES
# ==============================================================================

print("\n" + "=" * 80)
print("ANALISIS DE CORRELACIONES")
print("=" * 80)

# Extraer indices
nasdaq_index_data = df_final[df_final['nasdaq_10_index'].notna()].drop_duplicates('date').sort_values('date')
nasdaq_index = pd.Series(nasdaq_index_data['nasdaq_10_index'].values, index=nasdaq_index_data['date'])

crypto_index_data = df_final[df_final['crypto_7_index'].notna()].drop_duplicates('date').sort_values('date')
crypto_index = pd.Series(crypto_index_data['crypto_7_index'].values, index=crypto_index_data['date'])

# Fechas comunes
fechas_comunes = crypto_index.index.intersection(nasdaq_index.index)

if len(fechas_comunes) > 30:
    print(f"\nPeriodo analisis: {len(fechas_comunes):,} dias comunes")
    
    # Returns en fechas comunes
    crypto_returns = crypto_index.loc[fechas_comunes].pct_change().dropna()
    nasdaq_returns = nasdaq_index.loc[fechas_comunes].pct_change().dropna()
    
    # Correlacion
    correlacion = crypto_returns.corr(nasdaq_returns)
    print(f"\nCorrelacion CRYPTO-7 vs NASDAQ-10: {correlacion:.3f}")
    
    # Interpretacion
    if correlacion > 0.7:
        print("  Alta correlacion positiva - se mueven juntos")
    elif correlacion > 0.3:
        print("  Correlacion moderada - alguna sincronizacion")
    elif correlacion > -0.3:
        print("  Baja correlacion - movimientos independientes")
    else:
        print("  Correlacion negativa - movimientos opuestos")
    
    # Correlacion en estres
    stress_crypto = crypto_returns[crypto_returns < -0.05]
    stress_nasdaq = nasdaq_returns[nasdaq_returns < -0.05]
    
    print(f"\nANALISIS EN PERIODOS DE ESTRES:")
    print(f"Dias caida >5% CRYPTO: {len(stress_crypto)}")
    print(f"Dias caida >5% NASDAQ: {len(stress_nasdaq)}")
    
    if len(stress_crypto) + len(stress_nasdaq) > 5:
        all_stress = stress_crypto.index.union(stress_nasdaq.index)
        if len(all_stress) > 5:
            stress_corr = crypto_returns.loc[all_stress].corr(nasdaq_returns.loc[all_stress])
            print(f"Correlacion en estres: {stress_corr:.3f}")
else:
    print("\nDatos insuficientes para analisis de correlacion")


ANALISIS DE CORRELACIONES

Periodo analisis: 1,594 dias comunes

Correlacion CRYPTO-7 vs NASDAQ-10: 0.324
  Correlacion moderada - alguna sincronizacion

ANALISIS EN PERIODOS DE ESTRES:
Dias caida >5% CRYPTO: 118
Dias caida >5% NASDAQ: 12
Correlacion en estres: -0.025
